# Dax Options and Implied Volatilities

In this notebook we analyse DAX option prices.

In [ ]:
# Make sure you have all relevant Python packages.
# Missing packages can be installed with 'conda install ...' or 'pip install ...'

import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt # optional, if Plotly does not work

import QuantLib as ql

## Data preparation

Dax option data is published by Eurex, see e.g. https://www.eurex.com/ex-en/markets/idx/dax/70044!quotesSingleViewOption?callPut=Call&maturityDate=202212.

We read the option data from CSV file. The first 6 lines contain meta information (and comments) about dates and DAX underlying price. The remaining lines contain the actual option price data.

In [ ]:
header = pd.read_csv('DaxOptions.csv', nrows=4)
header

In [ ]:
data = pd.read_csv('DaxOptions.csv', skiprows=6)
data

In [ ]:
fig = go.Figure()
fig.update_layout(autosize=False, width=800, height=600, title='Option prices', xaxis_title='Strike price', yaxis_title='Price')
fig.add_scatter(x=data['Strike_Price'], y=data['Call_Ask'], name='Call_Ask', line=dict(color='darkred'),   mode='lines')
fig.add_scatter(x=data['Strike_Price'], y=data['Call_Bid'], name='Call_Bid', line=dict(color='red'),       mode='lines')
fig.add_scatter(x=data['Strike_Price'], y=data['Put_Ask'],  name='Put_Ask',  line=dict(color='blue'),      mode='lines')
fig.add_scatter(x=data['Strike_Price'], y=data['Put_Bid'],  name='Put_Bid',  line=dict(color='lightblue'), mode='lines')
fig.show()

If Plotly figures do not work in your environment you can uncomment below code and use Matplotlib for plotting.

In [ ]:
# plt.figure(figsize=(12,8))
# plt.title('Option prices')
# plt.xlabel('Strike price')
# plt.ylabel('Price')
# plt.plot(data['Strike_Price'], data['Call_Ask'], 'r-',  label='Call_Ask')
# plt.plot(data['Strike_Price'], data['Call_Bid'], 'r--', label='Call_Bid')
# plt.plot(data['Strike_Price'], data['Put_Ask'],  'b-',  label='Put_Ask')
# plt.plot(data['Strike_Price'], data['Put_Bid'],  'b--', label='Put_Bid')
# plt.legend()
# plt.show()

The data contain bid/ask prices for puts and calls for the Dec-22 expiry. This expiry represents the third Friday in December 2022, which is December 10, 2022.

Trade date is December 28, 2021 and initial DAX level ($S_0$) at 15.948,77. All prices are denominated in EUR.

We can also analyse bid-ask spreads of option prices:

In [ ]:
fig = go.Figure()
fig.update_layout(autosize=False, width=800, height=600, title='Bid/ask spreads', xaxis_title='Strike price', yaxis_title='Price')
fig.add_scatter(x=data['Strike_Price'], y=data['Call_Ask']-data['Call_Bid'], name='B/A Call', line=dict(color='red'),   mode='lines')
fig.add_scatter(x=data['Strike_Price'], y=data['Put_Ask'] -data['Put_Bid'],  name='B/A Put',  line=dict(color='blue'),  mode='lines')
fig.show()

In [ ]:
# plt.figure(figsize=(12,8))
# plt.title('Bid/ask spreads')
# plt.xlabel('Strike price')
# plt.ylabel('Price')
# plt.plot(data['Strike_Price'], data['Call_Ask']-data['Call_Bid'], 'r-', label='B/A Call')
# plt.plot(data['Strike_Price'], data['Put_Ask'] -data['Put_Bid'],  'b-', label='B/A Put')
# plt.legend()
# plt.show()

This view gives some nice insights about index option trading:

  - Bid/ask spreads for calls and puts are roughly comparable in the strike range from about 12.000 tro 19.000; This is probably the strike range where trading takes place.
  - For strikes below 11.500 and above 19.000 in-the-money options (low-strike calls and high-strike puts) show increasing bid/ask spreads; prices are probably not very liquid in that strike regions.

We want to analyse how above option prices translate into implied volatilities.

For risk-free rate calculation we consider mid prices. This makes things a bit simpler compared to using bid/ask for calls and puts.

In [ ]:
data['Call_Mid'] = 0.5 * (data['Call_Ask'] + data['Call_Bid'])
data['Put_Mid']  = 0.5 * (data['Put_Ask']  + data['Put_Bid'])
data

## Calculate Forward Index and Risk-free Rate from Put-Call Parity

- Consider $\pi^C(K)$ and $\pi^P(K)$ the (mid) call and put option price for a given strike $K$. 
- Apply put-call parity with $K_{ATM}=\mathbb{E}^*\left[ S_T \right]$. How can we calculate $\mathbb{E}^*\left[ S_T \right]$ model-independent from call and put option prices?
- Use the option data to numerically compute $\mathbb{E}^*\left[ S_T \right]$. Use e.g. *CubicSpline* and method *roots* from Scipy, https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.roots.html.

In [ ]:
# Add forward calculation here.

Use $K_{ATM}=\mathbb{E}^*\left[ S_T \right]$ to determine the effective continuous compounded risk-free rate.

- Use *ql.Date* to specify trading date and expiry date, https://quantlib-python-docs.readthedocs.io/en/latest/dates.html#date
- Use *ql.Actual365Fixed* DayCounter (https://quantlib-python-docs.readthedocs.io/en/latest/dates.html#daycounter) and method *yearFraction(d1,d2) to calculate time-to-expiry $T$ in years.
- Finally, use $S_0$, $K_{ATM}$ and $T$ to compute risk-free rate $r$ from the martingale condition
$$
  S_0 = \mathbb{E}^*\left[ \frac{S_T}{e^{rT}} \right].
$$


In [ ]:
# Add time to expiry calculation here

In [ ]:
# Add risk-free rate calculation here.

It turns out that the effective risk-free rate for Dec22 is about -15bp.

## Implied Volatility Calculation with QuantLib

Calculate implied volatilities for the quoted call bid/ask and put bid/ask prices. You need to setup a couple of QuantLib objects for this step.

  1.  You need a *ql.BlackScholesProcess*, https://quantlib-python-docs.readthedocs.io/en/latest/stochastic_processes.html. This takes the following inputs: 

      - *ql.SimpleQuote* to store market data values, and *ql.QuoteHandle* to allow for later update of quotes (not relevant for our example)

      - *ql.FlatForward* to store a risk-free rate curve with (trivial) business day calendar *ql.NullCalendar* and day count convention *ql.Actual365Fixed*

      - *ql.YieldTermStructureHandle* to allow for later replacement of a curve (not relevant for our example)

      - *ql.BlackConstantVol* to store volatility parameter with (trivial) business day calendar *ql.NullCalendar* and day count convention *ql.Actual365Fixed*

      - *ql.BlackVolTermStructureHandle*  to allow for later replacement of a volatility (not relevant for our example)

  2. You need to specify the option instruments, https://quantlib-python-docs.readthedocs.io/en/latest/instruments.html#vanilla-options

      - *ql.EuropeanExercise* to specify option expiry time
      
      - *ql.PlainVanillaPayoff* to specify call (*ql.Option.Call*) and put (*ql.Option.Put*) options for various strikes

  3. Finally, you can use the *option.impliedVolatility(price, process)* method for a given *option* instrument, input *price* and Black-Scholes *process*. This step needs a little care because not all prices allow for an implied volatility calculation. This point can be handled via try/except blocks, see function *safeImpliedVolatility*.

You can also follow the steps explained here:

https://stackoverflow.com/questions/4891490/calculating-europeanoptionimpliedvolatility-in-quantlib-python


In [ ]:
def safeImpliedVolatility(option, price, process):
    try:
        vol = option.impliedVolatility(price, process)
    except:
        vol = None
    return vol

In [ ]:
# Set evaluation date, setup market data and define stochastic process.

In [ ]:
# Setup options.

In [ ]:
# Calculate implied volatilities.
# Note: If implied volatilities cannot be calculated QuantLib throws an exception. You can handle that case via try-expcept blocks in Python.

Plot implied volatilities (and bid ask spreads) for calls and puts in terms of implied volatilities.

In [ ]:
# Adapt the option price plots to plot the resulting implied volatilities.

We find that:

  - Implied volatility spreads for in-the-money options are much wider then out-of-the money option spreads; this is because options are typically traded as protection at out-of-the-money strikes.
  - Around ATM, bid (and ask) volatilities for calls and puts match reasonably well; there are no arbitrage opportunities arising from put-call parity.